In [126]:
## Import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DayLocator, DateLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator
import numpy as np
import datetime, calendar
#from datetime import datetime
import matplotlib.patches as mpatches
%matplotlib inline

import os

## Import data frome pickle generated from muß ein file mit agentenstats sein
arcpth='/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/test_stats/archiv/'


In [127]:
def collectxlfiles(arcpath):
    xlfilelist=list()

    for xlfile in os.listdir(arcpath):
        if xlfile.startswith('CE_al'):
            xlfileabs=os.path.join(arcpath,xlfile)
            xlfilelist.append(xlfileabs)
    return sorted(xlfilelist)

In [128]:
xlfilelist=collectxlfiles(arcpth)
#xlfilelist
exfile=xlfilelist[0]

In [129]:

exframe=pd.read_excel(exfile) # this is a regular pd.DataFrame

In [130]:
agts_active=exframe.iloc[0, [1]].index.values.tolist()[0] # retrieve a string of all active agents (it's a string anyway)
exframe.columns=range(0,30) # rename columns to a temporarily more readable format, fancy rename later
#exframe.columns=exframe.loc[2,:].values # like so

In [131]:
extstamp=exframe.loc[3,0]
type(extstamp) # everything in here is a string (?)
exbearb=exframe.loc[3,4]
type(exbearb) # no it's not. int
exht=exframe.loc[3,[24,29]]
type(exht) # times are floats
exht

24    2.6833
29    2.6167
Name: 3, dtype: object

In [132]:

exframe

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,Startzeit,04.03 2017 : 00,UHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Endzeit,05.03 2017 : 00,UHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Timestamp,Agenten,Hotlines,angebotene Anrufe,bearbeitete Anrufe,bearbeitet ohne Wartezeit,bearbeitet nach Warteschleife,innerhalb 5 sek beantwortet,innerhalb 10 sek beantwortet,innerhalb 20 sek beantwortet,...,innerhalb 60 sek abgebrochen,innerhalb 120 sek abgebrochen,abgebrochene Anrufe,abgebrochene Anrufe (kostenlose Wartezeit),Gesamtbearbeitungszeit [min],durchschnittliche Bearbeitungszeit [sek],Durchschnittswartezeit bearbeiteter Anrufe [sek],Durchschnittswartezeit abgebrochener Anrufe [sek],Durchschnittswartezeit (kostenlos) abgebrochen...,Gesamtverbindungszeit [min]
3,04.03.2017 08:00,B tetzlva 613887,selected hotlines,1,1,1,0,0,0,1,...,0,0,0,0,2.6833,161,0,0,0,2.6167
4,04.03.2017 08:00,H beckfra 216694,selected hotlines,1,1,0,1,0,0,0,...,0,0,0,0,2.3667,142,54,0,0,2.1333
5,04.03.2017 09:00,B tetzlva 613887,selected hotlines,2,2,2,0,2,2,2,...,0,0,0,0,1.05,31.5,0,0,0,0.9167
6,04.03.2017 10:00,H scholan 189939,selected hotlines,1,1,0,1,0,0,0,...,0,0,0,0,2.1,126,31,0,0,0.6
7,04.03.2017 11:00,H dehnsil 386723,selected hotlines,1,1,0,1,0,0,0,...,0,0,0,0,3.4,204,30,0,0,2.4667
8,04.03.2017 11:00,H beckfra 216694,selected hotlines,1,1,1,0,0,1,1,...,0,0,0,0,4.3333,260,0,0,0,4.2333
9,04.03.2017 12:00,B tetzlva 613887,selected hotlines,3,3,3,0,3,3,3,...,0,0,0,0,6.5167,130.333,0,0,0,4.55
